In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[6]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.085725,3.438,0.0,0.0
1,22684.0,0.592652,3.217,0.0,0.0
2,21236.0,0.560293,3.103,0.0,0.0
3,20222.0,0.537633,3.051,0.0,0.0
4,19871.0,0.529789,2.982,0.0,0.0
...,...,...,...,...,...
43819,29347.0,0.741553,8.295,0.0,0.0
43820,27593.0,0.702355,7.805,0.0,0.0
43821,25226.0,0.649459,7.342,0.0,0.0
43822,23896.0,0.619737,6.959,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[6]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_ES,Temp_ES
0,0.0,3.438
1,22684.0,3.217
2,21236.0,3.103
3,20222.0,3.051
4,19871.0,2.982
...,...,...
43819,29347.0,8.295
43820,27593.0,7.805
43821,25226.0,7.342
43822,23896.0,6.959


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_ES,Temp_ES,Lag-2,Lag-7
0,25747.0,5.151,23909.0,0.0
1,24445.0,4.890,22087.0,22684.0
2,23847.0,4.670,20984.0,21236.0
3,23725.0,4.455,20487.0,20222.0
4,24384.0,4.295,20447.0,19871.0
...,...,...,...,...
43651,29347.0,8.295,28758.0,26307.0
43652,27593.0,7.805,28881.0,24646.0
43653,25226.0,7.342,27809.0,22481.0
43654,23896.0,6.959,25615.0,21518.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 14s 9ms/step - loss: 0.0137 - mape: 109400.7266 - mae: 0.0919 - val_loss: 0.0068 - val_mape: 10.4167 - val_mae: 0.0670
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0096 - mape: 106579.8828 - mae: 0.0802 - val_loss: 0.0046 - val_mape: 8.2896 - val_mae: 0.0537
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0058 - mape: 111991.8359 - mae: 0.0592 - val_loss: 0.0122 - val_mape: 13.3663 - val_mae: 0.0923
Epoch 4/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0043 - mape: 109424.4453 - mae: 0.0496 - val_loss: 0.0124 - val_mape: 13.8619 - val_mae: 0.0946
Epoch 5/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0034 - mape: 108898.8281 - mae: 0.0438 - val_loss: 0.0125 - val_mape: 13.8287 - val_mae: 0.0943
Epoch 6/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0031 - mape: 108655.3047 - mae: 0.0407 - val_l

Epoch 48/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0012 - mape: 101827.5781 - mae: 0.0238 - val_loss: 0.0095 - val_mape: 12.2780 - val_mae: 0.0806
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 98579.5469 - mae: 0.0232 - val_loss: 0.0125 - val_mape: 14.0046 - val_mae: 0.0923
Epoch 50/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 98733.3203 - mae: 0.0230 - val_loss: 0.0117 - val_mape: 13.4047 - val_mae: 0.0881
Epoch 51/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0011 - mape: 95374.6172 - mae: 0.0228 - val_loss: 0.0149 - val_mape: 15.3909 - val_mae: 0.1017
Epoch 52/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0011 - mape: 96787.1328 - mae: 0.0228 - val_loss: 0.0139 - val_mape: 14.8639 - val_mae: 0.0994
Epoch 53/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 94821.5781 - mae: 0.0227 - val_

1105/1105 [==============================] - 8s 7ms/step - loss: 6.9754e-04 - mape: 33470.4883 - mae: 0.0188 - val_loss: 0.0141 - val_mape: 15.2045 - val_mae: 0.0999
Epoch 95/100
1105/1105 [==============================] - 8s 7ms/step - loss: 7.2841e-04 - mape: 58730.8672 - mae: 0.0190 - val_loss: 0.0142 - val_mape: 15.2005 - val_mae: 0.0991
Epoch 96/100
1105/1105 [==============================] - 8s 7ms/step - loss: 7.7171e-04 - mape: 57280.1367 - mae: 0.0194 - val_loss: 0.0151 - val_mape: 15.8384 - val_mae: 0.1042
Epoch 97/100
1105/1105 [==============================] - 8s 7ms/step - loss: 6.6485e-04 - mape: 49072.8281 - mae: 0.0183 - val_loss: 0.0131 - val_mape: 14.6991 - val_mae: 0.0970
Epoch 98/100
1105/1105 [==============================] - 8s 7ms/step - loss: 6.9833e-04 - mape: 44399.4258 - mae: 0.0187 - val_loss: 0.0144 - val_mape: 15.4040 - val_mae: 0.1017
Epoch 99/100
1105/1105 [==============================] - 8s 7ms/step - loss: 6.6932e-04 - mape: 39531.4336 - mae: 0.0

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 864.4418270587921


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 9ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.5104737 , 0.50272423, 0.50041497, ..., 0.62319344, 0.58877695,
       0.5505075 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.6159562 , 0.60151056, 0.59801525, ..., 0.61659171, 0.58408291,
       0.56091122])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,25200.0,20884.501953
1,24609.0,20567.453125
2,24466.0,20472.976562
3,24763.0,21117.839844
4,26454.0,22680.912109
...,...,...
4363,29347.0,29426.578125
4364,27593.0,26826.121094
4365,25226.0,25496.089844
4366,23896.0,24088.042969


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

9.550343937060475